## Python, Boto3, and AWS S3

In [1]:
import boto3

Resource provides access to AWS services

In [2]:
s3_resource = boto3.resource('s3')

Display all buckets

In [6]:
for bucket in s3_resource.buckets.all():
    print(f'- {bucket.name}')

- dsl-smartkhata-backups
- image-service-deployment-a15dacef74
- image-service-dsl-bin-production
- pankaj-blog
- sp-test-03ebd6dd-efe7-4e49-aee5-a9d4a77fa19a
- sp-test-42072178-663e-4e5d-b57e-5b22f467e608


### Create s3 bucket

In [4]:
import uuid

def create_bucket_name(bucket_prefix):
    """
    Create a unique bucket name and add a prefix to specify what each bucket is for.
    """
    return ''.join([bucket_prefix, str(uuid.uuid4())])


def create_bucket(bucket_prefix, s3_resource):
    bucket_name = create_bucket_name(bucket_prefix)
    bucket_response = s3_resource.create_bucket(
        Bucket=bucket_name
    )
    return bucket_name, bucket_response

In [8]:
create_bucket_name('sp-test-')

'sp-test-2e13fb42-57a6-47d3-9d3f-795bcb8a4975'

In [7]:
bucket_name, bucket_response = create_bucket(bucket_prefix='sp-test-', s3_resource=s3_resource)

In [7]:
print(bucket_name)
print(bucket_response)

sp-test-42072178-663e-4e5d-b57e-5b22f467e608
s3.Bucket(name='sp-test-42072178-663e-4e5d-b57e-5b22f467e608')


In [8]:
for bucket in s3_resource.buckets.all():
    print(f'-{bucket.name}')

-dsl-smartkhata-backups
-image-service-deployment-a15dacef74
-image-service-dsl-bin-production
-pankaj-blog
-sp-test-03ebd6dd-efe7-4e49-aee5-a9d4a77fa19a
-sp-test-42072178-663e-4e5d-b57e-5b22f467e608


### Deleting S3 buckets

In [9]:
buckets_to_delete = [
    "sp-test-387b9fdd-92eb-40fe-918f-6027af13cd50",
    "sp-test-40ade20e-4504-49fd-840c-b7d8208d4766"
]

In [10]:
def delete_s3_buckets(buckets_to_delete):
    for bucket_name in buckets_to_delete:
        bucket= s3_resource.Bucket(bucket_name)
        
        # Check if the bucket exists
        if bucket.creation_date is None:
            print(f'Bucket {bucket_name} does not exist, skipping.')
            continue
            
        # Deleting all bucket's objects first
        for obj in bucket.objects.all():
            obj.delete()
                       
        # Now the bucket should be empty, so delete it
        try:
            bucket.delete()
            print(f"Deleted bucket {bucket_name}")
        except Exception as e:
            print(f"Error deleting bucket {bucket_name}: {e}")

In [11]:
delete_s3_buckets(buckets_to_delete)

Bucket sp-test-387b9fdd-92eb-40fe-918f-6027af13cd50 does not exist, skipping.
Bucket sp-test-40ade20e-4504-49fd-840c-b7d8208d4766 does not exist, skipping.


### Creating Bucket and Object Instances

In [12]:
file_name = 'first.txt'

In [13]:
first_bucket = s3_resource.Bucket(name=bucket_name)
# first_object = s3_resource.Object(bucket_name=bucket_name, key=file_name)
first_object = first_bucket.Object(file_name)

Uploading a file

In [14]:
first_object.upload_file(file_name)

### Traversals

Bucket traversal

In [15]:
for bucket in s3_resource.buckets.all():
    print(bucket.name)

dsl-smartkhata-backups
image-service-deployment-a15dacef74
image-service-dsl-bin-production
pankaj-blog
sp-test-03ebd6dd-efe7-4e49-aee5-a9d4a77fa19a
sp-test-42072178-663e-4e5d-b57e-5b22f467e608


Object traversal

In [16]:
for obj in first_bucket.objects.all():
    print(obj.key)

first.txt


In [17]:
for obj in first_bucket.objects.all():
    subsrc = obj.Object()
    print(obj.key, obj.storage_class, obj.last_modified)
    print(subsrc.version_id, subsrc.metadata)

first.txt STANDARD 2024-05-23 12:01:36+00:00
None {}


### Upload a file to S3 bucket

In [25]:
def upload_file_to_s3(file_path, bucket_name, object_name=None):
    if object_name is None:
        object_name = file_path.split('/')[-1]

    s3_resource = boto3.resource('s3')
    bucket = s3_resource.Bucket(bucket_name)

    try:
        bucket.upload_file(file_path, object_name)
        print(f"File '{file_path}' uploaded successfully to bucket '{bucket_name}' with key '{object_name}'")
    except Exception as e:
        print(f'Error uploading file to S3: {e}')
        
        
def upload_file_to_s3_using_client(file_path, bucket_name, object_name=None):
    # If object_name is not provided, use the local file name
    if object_name is None:
        object_name = file_path.split('/')[-1]
        
    # Create an S3 client
    s3_client = boto3.client('s3')
    
    try:
        s3_client.upload_file(file_path, bucket_name, object_name)
        print(f"File '{file_path}' uploaded successfully to bucket '{bucket_name}' with key '{object_name}'")
    except Exception as e:
        print(f'Error uploading file to S3: {e}')

In [19]:
bucket_name = 'sp-test-03ebd6dd-efe7-4e49-aee5-a9d4a77fa19a'

In [26]:
upload_file_to_s3('first.txt', bucket_name, 'folder/file.txt')
# Using file name as object name
upload_file_to_s3('new_schema.sql', bucket_name)

File 'first.txt' uploaded successfully to bucket 'sp-test-03ebd6dd-efe7-4e49-aee5-a9d4a77fa19a' with key 'folder/file.txt'
File 'new_schema.sql' uploaded successfully to bucket 'sp-test-03ebd6dd-efe7-4e49-aee5-a9d4a77fa19a' with key 'new_schema.sql'


### List all files in s3 bucket

In [27]:
def list_files_in_s3_bucket(bucket_name):
    s3_resource = boto3.resource('s3')
    s3_bucket = s3_resource.Bucket(bucket_name)
    
    files = [
        {
            'Key': obj.key,
            'Last Modified': obj.last_modified,
            'Size': obj.size
        }
        for obj in s3_bucket.objects.all()
    ]
    return files

In [28]:
list_files_in_s3_bucket(bucket_name)

[{'Key': 'firstfile.txt',
  'Last Modified': datetime.datetime(2024, 5, 23, 8, 48, 10, tzinfo=tzutc()),
  'Size': 9},
 {'Key': 'folder/file.txt',
  'Last Modified': datetime.datetime(2024, 5, 23, 12, 11, 1, tzinfo=tzutc()),
  'Size': 9},
 {'Key': 'new_schema.sql',
  'Last Modified': datetime.datetime(2024, 5, 23, 12, 11, 2, tzinfo=tzutc()),
  'Size': 940}]

### Download file from s3

In [23]:
def download_file_from_s3(download_path, bucket_name, object_name):
    s3_client = boto3.client('s3')
    
    try:
        s3_client.download_file(bucket_name, object_name, download_path)
        print(f"File '{object_name}' downloaded successfully from bucket '{bucket_name}' to '{download_path}'")
    except Exception as e:
        print(f'Error downloading file from S3: {e}')

In [24]:
download_file_from_s3('./downloaded_file.txt', bucket_name, 'firstfile.txt')

File 'firstfile.txt' downloaded successfully from bucket 'sp-test-03ebd6dd-efe7-4e49-aee5-a9d4a77fa19a' to './downloaded_file.txt'
